In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import re
import pyinputplus as pyip
import requests
import datetime as dt
from datetime import datetime
import pickle
import random
import math
import string

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

In [358]:
# Constants

YDS_GRADES_FULL = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.7+', '5.8-', '5.8', '5.8+', '5.9-', '5.9', '5.9+', '5.10a', '5.10-', '5.10a/b', '5.10b', '5.10', '5.10b/c', '5.10c', '5.10+', '5.10c/d', '5.10d', '5.11a', '5.11-', '5.11a/b', '5.11b', '5.11', 
                   '5.11b/c', '5.11c', '5.11+', '5.11c/d', '5.11d', '5.12a', '5.12-', '5.12a/b', '5.12b', '5.12', '5.12b/c', '5.12c', '5.12+', '5.12c/d', '5.12d', '5.13a', '5.13-', '5.13a/b', '5.13b', '5.13', '5.13b/c', '5.13c', '5.13+', '5.13c/d', '5.13d', '5.14a', 
                   '5.14-', '5.14a/b', '5.14b', '5.14', '5.14b/c', '5.14c', '5.14+', '5.14c/d', '5.14d', '5.15a', '5.15-', '5.15a/b', '5.15b', '5.15', '5.15b/c' '5.15c', '5.15+', '5.15c/d', '5.15d'] 
YDS_GRADES_LETTER = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '5.10a', '5.10b', '5.10c', '5.10d', '5.11a', '5.11b', '5.11c', '5.11d', '5.12a', '5.12b', '5.12c', '5.12d', '5.13a', '5.13b', '5.13c', '5.13d', '5.14a', '5.14b', '5.14c', '5.14d', '5.15a', '5.15b', '5.15c', '5.15d']
YDS_GRADES_SIGN = ['5.0', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.7+', '5.8-', '5.8', '5.8+', '5.9-', '5.9', '5.9+', '5.10-', '5.10', '5.10+', '5.11-', '5.11', '5.11+', '5.12-', '5.12', '5.12+', '5.13-', '5.13', '5.13+', '5.14-', '5.14', '5.14+', '5.15-', '5.15', '5.15+']

V_GRADES_FULL = ['V-easy', 'V0-', 'V0', 'V0+', 'V0-1', 'V1-', 'V1', 'V1+', 'V1-2', 'V2-', 'V2', 'V2+', 'V2-3', 'V3-', 'V3', 'V3+', 'V3-4', 'V4-', 'V4', 'V4+', 'V4-5', 'V5-', 'V5', 'V5+', 'V5-6', 'V6-', 'V6', 'V6+', 'V6-7', 'V7-', 'V7', 'V7+', 'V7-8', 'V8-', 'V8', 'V8+', 'V8-9', 
                 'V9-', 'V9', 'V9+', 'V9-10', 'V10-', 'V10', 'V10+', 'V10-11', 'V11-', 'V11', 'V11+', 'V11-12', 'V12-', 'V12', 'V12+', 'V12-13', 'V13-', 'V13', 'V13+', 'V13-14', 'V14-', 'V14', 'V14+', 'V14-15', 'V15-', 'V15', 'V15+', 'V15-16', 'V16-', 'V16', 'V16+', 'V16-17', 'V17-', 'V17']
V_GRADES_SIGN = ['V-easy', 'V0-', 'V0', 'V0+', 'V1-', 'V1', 'V1+', 'V2-', 'V2', 'V2+', 'V3-', 'V3', 'V3+', 'V4-', 'V4', 'V4+', 'V5-', 'V5', 'V5+', 'V6-', 'V6', 'V6+', 'V7-', 'V7', 'V7+', 'V8-', 'V8', 'V8+', 
                 'V9-', 'V9', 'V9+', 'V10-', 'V10', 'V10+', 'V11-', 'V11', 'V11+', 'V12-', 'V12', 'V12+', 'V13-', 'V13', 'V13+', 'V14-', 'V14', 'V14+', 'V15-', 'V15', 'V15+', 'V16-', 'V16', 'V16+', 'V17-', 'V17']
V_GRADES_BASIC = ['V-easy', 'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17']

RISK_GRADES = ['PG', 'PG13', 'R', 'X']

CLEAN_SEND = ['onsight', 'flash', 'redpoint', 'pinkpoint', 'send']
CLEAN_SEND_FIRST = ['onsight', 'flash']
CLEAN_SEND_WORKED = ['redpoint', 'pinkpoint']
TICK_OPTIONS = ['solo', 'tr', 'follow', 'lead', 'fell/hung', 'onsight', 'flash', 'redpoint', 'pinkpoint', 'send', 'attempt']

Allow user to import ticks, then perform initial data formatting and handling.

In [378]:
# https://www.mountainproject.com/user/200180658/brayden-l
try:
    resp = input('input link to profile of format "https://www.mountainproject.com/user/..."\n')
    df_usend = pd.read_csv(f'{resp}/tick-export')
except Exception as err:
    print(err)

In [379]:
# Transform all strings to lower case so we don't have to worry about it for operations. Can be turned to camel case later for presentation.
df_usend[['Style', 'Lead Style', 'Route Type']] = df_usend[['Style', 'Lead Style', 'Route Type']].applymap(lambda s: s.lower() if type(s)==str else s)

# 'Date' to datetype
df_usend['Date'] = pd.to_datetime(df_usend['Date'])

# Remove all aid, ice, snow, TR only,and trad/boulder climbing route types as they are not relevant.
df_usend = df_usend[df_usend['Route Type'].str.contains(r'aid|ice|snow') != True]
df_usend = df_usend[df_usend['Route Type'].str.fullmatch(r'tr') != True] #if this is just a partial match it will detech "trad" too!
df_usend = df_usend[df_usend['Route Type'].str.contains(r'trad') & df_usend['Route Type'].str.contains(r'boulder') != True]

# "trad, sport" goes to "trad". If it uses gear it's trad!
df_usend.loc[df_usend['Route Type'].str.contains(r'trad') & df_usend['Route Type'].str.contains(r'sport'), 'Route Type'] = 'trad'

# "x, alpine" and "x, tr" goes to "x" Alpine and tr tags are not useful.
def rem_route_el_from_list(ousted, seperator):
    el_rem_subset = df_usend['Route Type'].str.contains(ousted) == True
    df_usend.loc[el_rem_subset, 'Route Type'] = df_usend[el_rem_subset]['Route Type'].apply(lambda row: [val for val in row.split(seperator) if val != ousted]).apply(lambda x: ", ".join(x))

rem_route_el_from_list('alpine', ', ')
rem_route_el_from_list('tr', ', ')

# Extract route unique identifier from URL and create a new column for it.
df_usend.insert(len(df_usend.columns),'Route ID','')
df_usend['Route ID'] = df_usend['URL'].apply(lambda x: x.split('/')[4])

# Change YDS-Vgrade combos to just Vgrade. They are most likely boulders, so a bouldering grade is relevant.
subset = df_usend['Rating'].apply(lambda row: [val for val in row.split() if val in V_GRADES_FULL]).astype(bool)  & df_usend['Rating'].apply(lambda row: [val for val in row.split() if val in YDS_GRADES_FULL]).astype(bool) == True
df_usend.loc[subset, 'Rating'] = df_usend[subset]['Rating'].apply(lambda x: x.split()[1])

# Seperate risk rating to new column
df_usend.insert(df_usend.columns.get_loc('Rating')+1,'Risk','')
df_usend['Risk'] = df_usend['Rating'].apply(lambda row: [val for val in row.split() if val in RISK_GRADES]).apply(lambda x: "".join(x))
# Reduce Rating column to just rating
df_usend['Rating'] = df_usend['Rating'].apply(lambda row: [val for val in row.split()][0])
# Displays all rows where risk ratings have been seperated
# df_usend[df_usend['Risk'].astype(bool)]

# Create original rating and length archive to compare against or undo changes.
df_usend.insert(df_usend.columns.get_loc('Rating'),'Original Rating',df_usend['Rating'])

# Apostrophe's are html coded in the notes section for some reason.
df_usend['Notes'] = df_usend['Notes'].apply(lambda x: str(x).replace('&#39;',"'"))

df_usend.insert(len(df_usend.columns),'Flash/Onsight','')
df_usend.insert(len(df_usend.columns),'Worked Clean','')
df_usend.insert(len(df_usend.columns),'Grade Breakthrough','')
df_usend.insert(len(df_usend.columns),'Attempts','')

Perform scrape and analytics on each unique climb

In [ ]:
# Download page of each unique climb and export to a unique dataframe

df_usend_uniq = df_usend.drop_duplicates(subset="Route ID")
df_usend_uniq.drop(columns=['Date', 'Notes', 'Pitches', 'Your Stars', 'Style', 'Lead Style', 'Your Rating', 'Flash/Onsight', 'Worked Clean', 'Grade Breakthrough', 'Attempts'], inplace=True)
df_usend_uniq.insert(len(df_usend_uniq.columns),'Listed Pitches','')
df_usend_uniq.insert(len(df_usend_uniq.columns),'re mainpage','')
df_usend_uniq.insert(len(df_usend_uniq.columns),'re statpage','')
df_usend_uniq.insert(len(df_usend_uniq.columns),'Statpage df','')
df_usend_uniq.insert(len(df_usend_uniq.columns),'Tick Counts','')

def insert_str_to_address(url, insert_phrase):
    str_list = url.split('/')
    str_list.insert(4, insert_phrase)
    return '/'.join(str_list)

def page_download(url):
    try:
        res = requests.get(url, timeout=10)
    except Exception as e:
        print(e)
    print(res)
    return res

df_usend_uniq['re mainpage'] = df_usend_uniq['URL'].apply(page_download)
df_usend_uniq['re statpage'] = df_usend_uniq['URL'].apply(lambda x: page_download(insert_str_to_address(x, 'stats')))

In [382]:
# Let's analyze the mainpage for listed default pitch lengths. This is not included in the tick export but is helpful in determining user tick methodology

def get_pitches(res):
    soup = BeautifulSoup(res.text, 'html.parser')
    route_type_text = str(soup.find(class_="description-details").find_all('td')[1])
    pitch_search = re.search(r'\d+ pitches',route_type_text)
    if str(type(pitch_search)) == "<class 'NoneType'>":
        num_pitches = 1
    else:
        num_pitches = pitch_search.group(0).split(' ')[0]
    return int(num_pitches)

df_usend_uniq['Listed Pitches'] = df_usend_uniq['re mainpage'].apply(get_pitches)


C:\Users\Brayden\AppData\Local\Temp\ipykernel_13844\537939784.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Extract tick details

def get_tick_details(res):
    name = []
    namelink = []
    entrydate = []
    pitches = []
    style = []
    lead_style = []
    comment = []
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # print(soup.select("#route-stats > div.row.pt-main-content > div > h1")) # Tells you which page is being scraped, useful for debugging
    try:
        blocks = list(soup.select("#route-stats > div:nth-child(2) > div:nth-last-child(1)")[0].find_all('tr'))
    except:
        blocks = []
    for x in blocks:
        soup = BeautifulSoup(str(x), 'html.parser')
        entries = soup.find_all('div', attrs={'class': None})
        for entry in entries:
            entrytext = entry.text
            try:
                name.append(soup.find('a').text.strip())
            except:
                name.append('')
                
            try:
                namelink.append(soup.find('a')['href'].strip())
            except:
                namelink.append('')
            
            try:
                date_search = [re.search(r'\w{3}\s\d{1,2},\s\d{4}', entrytext)]
                entrydate.append([subresult.group(0).strip() if subresult else '' for subresult in date_search][0]) # pulls match text if match object is not none
            except:
                entrydate.append('')
            
            try:
                pitches_search = [re.search(r'·([^.]+\s(pitches))', entrytext)] # regex for starting at · and ending at first period only if it includes the word "pitches"
                pitchesinterm = [subresult.group(0) if subresult else '' for subresult in pitches_search]
                pitches.append([int(re.search(r'\d+', subresult).group(0).strip()) if subresult else 1 for subresult in pitchesinterm][0]) # take just the digit of the string
            except:
                pitches.append(1)
            
            try:
                style_search = [re.search(r"(solo|tr|follow|lead|send|attempt|flash)", entrytext.lower())]
                style_val = [subresult.group(0).strip() if subresult else '' for subresult in style_search][0] # I have a conditional in the comment search that depends on this so I made it a separate variable
                style.append(style_val)
            except:
                style.append('')
            
            try:
                if style_val != '':
                    lead_style_search = [re.search(r"/([^.]+)", entrytext.lower())]
                    lead_style.append([subresult.group(0)[2:].strip() if subresult else '' for subresult in lead_style_search][0])
                else:
                    lead_style.append('')
            except:
                lead_style.append('')
            
            try:
                if style_val != '': 
                    comment_search = [re.search(r"(Solo|TR|Follow|Lead).*", entrytext)]
                    commentinterm = ([subresult.group(0) if subresult else '' for subresult in comment_search])
                    comment.append([re.search(r"\..*", subresult).group(0)[2:].strip() if subresult else '' for subresult in commentinterm][0])
                else:
                    comment_search = [re.search(r"·(.*)", entrytext)] # If no style comment then entire phrase is the comment.
                    comment.append([subresult.group(0)[2:].strip() if subresult else '' for subresult in comment_search][0])
            except:
                comment.append('')
    # print (len(name),len(namelink),len(entrydate),len(pitches),len(style),len(lead_style),len(comment))
    # print (name,namelink,entrydate,pitches,style,lead_style,comment)
    d = pd.DataFrame({'Username' : name, "User Link" : namelink, "Entry Date": entrydate, "Pitches": pitches, "Style": style, "Lead Style": lead_style, "Comment": comment})
    # One last possible error correction, an oomlot injected a "/" into lead style and the regex incidentally detected it
    d.loc[~d['Lead Style'].isin(TICK_OPTIONS), 'Lead Style'] = ''
    return d

df_usend_uniq['Statpage df']=df_usend_uniq['re statpage'].apply(get_tick_details)


In [ ]:
# Analyze tick details

def unpack_style(statpage_df, colref):
    nest_list = []
    for row in statpage_df.index:
        styleval = statpage_df[colref][row]
        if styleval in CLEAN_SEND: # clean sends with multiple ticks are assumed to be fell/hung attempts up to that clean send.
            nest_list.append([statpage_df[colref][row]])
            nest_list.append((statpage_df['Pitches'][row]-1)*['fell/hung'])
        else:
            nest_list.append(statpage_df['Pitches'][row]*[statpage_df[colref][row]])
    flat_list = [num for sublist in nest_list for num in sublist]
    return flat_list

def analyze_tick_counts(statpagedf):
    search_counts_init = pd.Series(np.nan, index=TICK_OPTIONS).to_frame()
    tick_type_list = unpack_style(statpagedf, 'Style') + unpack_style(statpagedf, 'Lead Style')
    tick_type_list = list(filter(None, tick_type_list))
    search_counts = pd.Series(tick_type_list).value_counts().dropna().to_frame()
    search_counts = search_counts_init.fillna(search_counts).fillna(0).squeeze()
    lead_ratio = search_counts['lead']/(search_counts['follow'] + search_counts['tr'] + search_counts['lead'])
    os_ratio = (search_counts['onsight'] + search_counts['flash']) / (search_counts['onsight'] + search_counts['flash'] + search_counts['fell/hung'] + search_counts['redpoint'] + search_counts['pinkpoint'] + search_counts['attempt'] + search_counts['send'])
    return pd.Series([lead_ratio, os_ratio])

df_usend_uniq[['Lead Ratio', 'OS Ratio']] = df_usend_uniq['Statpage df'].apply(analyze_tick_counts)

In [385]:
# Let's save the now scraped dataframe to a pickle file
df_usend_uniq.to_pickle('../Data_Archive/df_archive_base')

In [130]:
# Load the pickle file
picklefile = open('../Data_Archive/df_archive_base', 'rb')
df_usend_uniq = pickle.load(picklefile)

Now we further clean up our data set.

In [386]:
# Define roped and bouldering specific subset
roped_subset = (df_usend_uniq['Route Type'] == 'sport') | (df_usend_uniq['Route Type'] == 'trad')
boulder_subset = (df_usend_uniq['Route Type'] == 'boulder')

In [ ]:
# Handle route length outliers
ROPED_MIN_LENGTH = 15
ROPED_MAX_LENGTH = 4500 #"Trango Towers" are 4,300' tall
BOULDER_MIN_LENGTH = 1
BOULDER_MAX_LENGTH = 55 #"Too Tall to Fall" is 50'

# Fix outliers
def fix_length_outliers(dataframe, subset, minlength, maxlength):
    length_outliers = dataframe[subset][(dataframe[subset]['Length'] <= minlength) | (dataframe[subset]['Length'] >= maxlength)]
    for loop_count, (index, data) in enumerate(length_outliers.iterrows()):
        updated_length = pyip.inputNum(f"[{loop_count+1}/{length_outliers.shape[0]}] Outlier Detected, Possible Bad Info. Input Corrected Length for\nRoute: {data['Route']}\nLocation: {'>'.join(data['Location'].split('>')[-3:])}\nCurrently: {data['Length']}ft\n", min=minlength, max=maxlength)
        dataframe.at[index, 'Length'] = updated_length
    return dataframe

# Fill empty route lengths
def fill_length_empties(dataframe, subset, minlength, maxlength):
    length_missing = dataframe[subset][(dataframe[subset]['Length'].isnull()) | (dataframe[subset]['Length'] == 0)]
    for loop_count, (index, data) in enumerate(length_missing.iterrows()):
        updated_length = pyip.inputNum(f"[{loop_count+1}/{length_missing.shape[0]}] Input Estimated Length for\nRoute: {data['Route']}\nLocation: {'>'.join(data['Location'].split('>')[-3:])}\n", min=minlength, max=maxlength)
        dataframe.at[index, 'Length'] = updated_length
    return dataframe

df_usend_uniq = fix_length_outliers(df_usend_uniq, roped_subset, ROPED_MIN_LENGTH, ROPED_MAX_LENGTH)
df_usend_uniq = fill_length_empties(df_usend_uniq, roped_subset, ROPED_MIN_LENGTH, ROPED_MAX_LENGTH)
df_usend_uniq = fix_length_outliers(df_usend_uniq, boulder_subset, BOULDER_MIN_LENGTH, BOULDER_MAX_LENGTH)
df_usend_uniq = fill_length_empties(df_usend_uniq, boulder_subset, BOULDER_MIN_LENGTH, BOULDER_MAX_LENGTH)

#TODO allow user to select "assign median height from area"
#TODO check if route length is an outlier for area in addition to "globally" an outlier. Might be more accurate.

In [405]:
# Homogenize Grades
grade_homo_choice = ['round down', 'round up', 'round evenly', 'hand determine']
rating_isolate = df_usend_uniq['Original Rating'].apply(lambda row: [val for val in row.split()][0])

# Reset 'Rating' column so this mapping can be re-run
df_usend_uniq["Rating"] = df_usend_uniq["Original Rating"]

#Roped Grades
print('ROPED GRADES')
print('Select Roped Grade Homogenization Type:\n1)5.10a / 5.10b / 5.10c / 5.10d (Requires additional effort)\n2)5.10- / 5.10 / 5.10+')
r_grade_homo_type = pyip.inputChoice(['1','2'])
if r_grade_homo_type == '1':
    r_grade_homo_dir = pyip.inputChoice(grade_homo_choice)

rgrademoderatemap = {'5.6-': '5.6', '5.6+': '5.6', '5.7-': '5.7', '5.7+': '5.7', '5.8-': '5.8', '5.8+': '5.8', '5.9-': '5.9', '5.9+': '5.9'}
rgradedownmap = {'5.10a/b': '5.10a', '5.10-': '5.10a', '5.10b/c': '5.10b', '5.10': '5.10b', '5.10c/d': '5.10c', '5.10+': '5.10c',
            '5.11a/b': '5.11a', '5.11-': '5.11a', '5.11b/c': '5.11b', '5.11': '5.11b', '5.11c/d': '5.11c', '5.11+': '5.11c',
            '5.12a/b': '5.12a', '5.12-': '5.12a', '5.12b/c': '5.12b', '5.12': '5.12b', '5.12c/d': '5.12c', '5.12+': '5.12c',
            '5.13a/b': '5.13a', '5.13-': '5.13a', '5.13b/c': '5.13b', '5.13': '5.13b', '5.13c/d': '5.13c', '5.13+': '5.13c',
            '5.14a/b': '5.14a', '5.14-': '5.14a', '5.14b/c': '5.14b', '5.14': '5.14b', '5.14c/d': '5.14c', '5.14+': '5.14c',
            '5.15a/b': '5.15a', '5.15-': '5.15a', '5.15b/c': '5.15b', '5.15': '5.15b', '5.15c/d': '5.15c', '5.15+': '5.15c',
}
rgradeupmap = {'5.10a/b': '5.10b', '5.10-': '5.10b', '5.10b/c': '5.10c', '5.10': '5.10c', '5.10c/d': '5.10d', '5.10+': '5.10d',
            '5.11a/b': '5.11b', '5.11-': '5.11b', '5.11b/c': '5.11c', '5.11': '5.11c', '5.11c/d': '5.11d', '5.11+': '5.11d',
            '5.12a/b': '5.12b', '5.12-': '5.12b', '5.12b/c': '5.12c', '5.12': '5.12c', '5.12c/d': '5.12d', '5.12+': '5.12d',
            '5.13a/b': '5.13b', '5.13-': '5.13b', '5.13b/c': '5.13c', '5.13': '5.13c', '5.13c/d': '5.13d', '5.13+': '5.13d',
            '5.14a/b': '5.14b', '5.14-': '5.14b', '5.14b/c': '5.14c', '5.14': '5.14c', '5.14c/d': '5.14d', '5.14+': '5.14d',
            '5.15a/b': '5.15b', '5.15-': '5.15b', '5.15b/c': '5.15c', '5.15': '5.15c', '5.15c/d': '5.15d', '5.15+': '5.15d',
}
rgradecompmap = {'5.10a': '5.10-', '5.10a/b': '5.10-', '5.10b': '5.10', '5.10c':'5.10', '5.10b/c': '5.10', '5.10d': '5.10+', '5.10c/d': '5.10+',
            '5.11a': '5.11-', '5.11a/b': '5.11-', '5.11b': '5.11','5.11c':'5.10', '5.11b/c': '5.11', '5.11d': '5.11+', '5.11c/d': '5.11+',
            '5.12a': '5.12-', '5.12a/b': '5.12-', '5.12b': '5.12','5.12c':'5.10', '5.12b/c': '5.12', '5.12d': '5.12+', '5.12c/d': '5.12+',
            '5.13a': '5.13-', '5.13a/b': '5.13-', '5.13b': '5.13','5.13c':'5.10', '5.130b/c': '5.13', '5.13d': '5.13+', '5.13c/d': '5.13+',
            '5.14a': '5.14-', '5.14a/b': '5.14-', '5.14b': '5.14','5.14c':'5.10', '5.14b/c': '5.14', '5.14d': '5.14+', '5.14c/d': '5.14+',
}

def grademoderate():
    grade_change_subset = rating_isolate.isin(list(rgrademoderatemap.keys()))
    df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq.loc[grade_change_subset]['Original Rating'].map(rgrademoderatemap)

def grade_split(upmap, downmap):
    grade_change_subset = rating_isolate.isin(list(upmap.keys()))
    grade_change_subset_df = df_usend_uniq[grade_change_subset]
    for grade in grade_change_subset_df['Original Rating'].unique():
        to_change = grade_change_subset_df[grade_change_subset_df['Original Rating'] == grade]
        changed_up = to_change.sample(frac=0.5)['Original Rating'].map(upmap)
        df_usend_uniq.loc[changed_up.index, 'Rating'] = changed_up
    grade_change_subset = rating_isolate.isin(list(downmap.keys()))
    grade_change_subset_df = df_usend_uniq[grade_change_subset]
    for grade in grade_change_subset_df['Original Rating'].unique():
        to_change = grade_change_subset_df[grade_change_subset_df['Original Rating'] == grade]
        changed_down = to_change['Original Rating'].map(downmap)
        df_usend_uniq.loc[changed_down.index, 'Rating'] = changed_down

if r_grade_homo_type == '2':
    grade_change_subset = rating_isolate.isin(list(rgradecompmap.keys()))
    df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(rgradecompmap)
else:
    if r_grade_homo_dir == grade_homo_choice[0]:
        grademoderate()
        grade_change_subset = rating_isolate.isin(list(rgradedownmap.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(rgradedownmap)
    if r_grade_homo_dir == grade_homo_choice[1]:
        grademoderate()
        grade_change_subset = rating_isolate.isin(list(rgradeupmap.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(rgradeupmap)
    if r_grade_homo_dir == grade_homo_choice[2]:
        grademoderate()
        grade_split(rgradeupmap,rgradedownmap)
    if r_grade_homo_dir == grade_homo_choice[3]:
        needs_grade_corr = df_usend_uniq[rating_isolate.isin(list(rgrademoderatemap.keys()) + list(rgradedownmap.keys()))]
        for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=YDS_GRADES_LETTER)
            df_usend_uniq.at[index, 'Rating'] = updated_grade

#Boulder Grades
print('BOULDER GRADES')
print('Select Boulder Grade Homogenization Type:\n1)V1\n2)V1- / V1 / V1+')
b_grade_homo_type = pyip.inputChoice(['1','2'])
b_grade_homo_dir = pyip.inputChoice(grade_homo_choice)

#1 type
bgradedownmap1 = {'V0-1': 'V0', 'V1-2': 'V1', 'V2-3': 'V2', 'V3-4': 'V3', 'V4-5': 'V4', 'V5-6': 'V5', 'V6-7': 'V6', 'V7-8': 'V7', 'V8-9': 'V8', 'V9-10': 'V9', 'V10-11': 'V10', 'V11-12': 'V11', 'V12-13': 'V12', 'V13-14': 'V13', 'V14-15': 'V14', 'V15-16':'V15', 'V16-17': 'V16'}
bgradeupmap1 = {'V0-1': 'V1', 'V1-2': 'V2', 'V2-3': 'V3', 'V3-4': 'V4', 'V4-5': 'V5', 'V5-6': 'V6', 'V6-7': 'V7', 'V7-8': 'V8', 'V8-9': 'V9', 'V9-10': 'V10', 'V10-11': 'V11', 'V11-12': 'V12', 'V12-13': 'V13', 'V13-14': 'V14', 'V14-15': 'V15', 'V15-16':'V16', 'V16-17': 'V17'}
bgradeconmap1 = {'V0-': 'V0', 'V0+': 'V0', 'V1-': 'V1', 'V1+': 'V1', 'V2-': 'V2', 'V2+': 'V2', 'V3-': 'V3', 'V3+': 'V3', 'V4-': 'V4', 'V4+': 'V4', 'V5-': 'V5', 'V5+': 'V5', 'V6-': 'V6', 'V6+': 'V6', 'V7-': 'V7', 'V7+': 'V7', 'V8-': 'V8', 'V8+': 'V8', 
                'V9-': 'V9', 'V9+': 'V9', 'V10-': 'V10', 'V10+': 'V10', 'V11-': 'V11', 'V11+': 'V11', 'V12-': 'V12', 'V12+': 'V12', 'V13-': 'V13', 'V13+': 'V13', 'V14-': 'V14', 'V14+': 'V14', 'V15-': 'V15', 'V15+': 'V15', 'V16-': 'V16', 'V16+': 'V16', 'V17-': 'V17', 'V17+': 'V17'}
#2 type
bgradedownmap2 = {'V0-1': 'V0+', 'V1-2': 'V1+', 'V2-3': 'V2+', 'V3-4': 'V3+', 'V4-5': 'V4+', 'V5-6': 'V5+', 'V6-7': 'V6+', 'V7-8': 'V7+', 'V8-9': 'V8+', 'V9-10': 'V9+', 'V10-11': 'V10+', 'V11-12': 'V11+', 'V12-13': 'V12+', 'V13-14': 'V13+', 'V14-15': 'V14+', 'V15-16':'V15+', 'V16-17': 'V16+'}
bgradeupmap2 = {'V0-1': 'V1-', 'V1-2': 'V2-', 'V2-3': 'V3-', 'V3-4': 'V4-', 'V4-5': 'V5-', 'V5-6': 'V6-', 'V6-7': 'V7-', 'V7-8': 'V8-', 'V8-9': 'V9-', 'V9-10-': 'V10', 'V10-11': 'V11-', 'V11-12': 'V12-', 'V12-13': 'V13-', 'V13-14': 'V14-', 'V14-15': 'V15-', 'V15-16':'V16-', 'V16-17': 'V17-'}

if b_grade_homo_type == '1':
    # Remove all + and - grades
    grade_change_subset = rating_isolate.isin(list(bgradeconmap1.keys()))
    df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(bgradeconmap1)

    if b_grade_homo_dir == grade_homo_choice[0]:
        grade_change_subset = rating_isolate.isin(list(bgradedownmap1.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(bgradedownmap1)
    if b_grade_homo_dir == grade_homo_choice[1]:
        grade_change_subset = rating_isolate.isin(list(bgradeupmap1.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(bgradeupmap1)
    if b_grade_homo_dir == grade_homo_choice[2]:
        grade_split(bgradeupmap1,bgradedownmap1)
    if b_grade_homo_dir == grade_homo_choice[3]:
        needs_grade_corr = df_usend_uniq[rating_isolate.isin(list(bgradedownmap1.keys()))]
        for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=V_GRADES_BASIC)
            df_usend_uniq.at[index, 'Rating'] = updated_grade

if b_grade_homo_type =='2':
    if b_grade_homo_dir == grade_homo_choice[0]:
        grade_change_subset = rating_isolate.isin(list(bgradedownmap2.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(bgradedownmap2)
    if b_grade_homo_dir == grade_homo_choice[1]:
        grade_change_subset = rating_isolate.isin(list(bgradeupmap2.keys()))
        df_usend_uniq.loc[grade_change_subset, 'Rating'] = df_usend_uniq[grade_change_subset]['Original Rating'].map(bgradeupmap2)
    if b_grade_homo_dir == grade_homo_choice[2]:
        grade_split(bgradeupmap2,bgradedownmap2)
    if b_grade_homo_dir == grade_homo_choice[3]:
        needs_grade_corr = df_usend_uniq[rating_isolate.isin(list(bgradedownmap2.keys()))]
        for loop_count, (index, data) in enumerate(needs_grade_corr.iterrows()):
            updated_grade = pyip.inputChoice(prompt=f"[{loop_count+1}/{needs_grade_corr.shape[0]}] Input Grade Correction For: {data['Route'].title()}:\n", choices=V_GRADES_BASIC)
            df_usend_uniq.at[index, 'Rating'] = updated_grade

C:\Users\Brayden\AppData\Local\Temp\ipykernel_13844\1500877131.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ROPED GRADES
Select Roped Grade Homogenization Type:
1)5.10a / 5.10b / 5.10c / 5.10d (Requires additional effort)
2)5.10- / 5.10 / 5.10+
Please select one of: 1, 2
Please select one of: round down, round up, round evenly, hand determine
BOULDER GRADES
Select Boulder Grade Homogenization Type:
1)V1
2)V1- / V1 / V1+
Please select one of: 1, 2
Please select one of: round down, round up, round evenly, hand determine


In [406]:
# merge unique dataframe details to user data frame. This will delete the user data frame of length and rating information and replace it with that from the unique dataframe
df_usendm = df_usend.copy() # We create a copy of the original df_usend to be consequently modified by the user. This leaves the original dataframe alone.
df_usendm.drop(columns=['Rating', 'Length'], inplace=True)
df_usendm = df_usendm.merge(df_usend_uniq[['Route ID', 'Listed Pitches', 'Lead Ratio', 'OS Ratio', 'Rating', 'Length']], how='left', on='Route ID')
df_usendm = df_usendm[['Date', 'Route', 'Original Rating', 'Rating', 'Risk', 'Notes', 'URL', 'Pitches', 'Listed Pitches', 'Location', 'Avg Stars', 'Your Stars', 'Style', 'Lead Style', 'Route Type', 'Length', 'Rating Code','Route ID', 'Flash/Onsight', 'Grade Breakthrough', 'Worked Clean', 'Attempts', 'Lead Ratio', 'OS Ratio']] # Reorder

In [408]:
# We want to tag important climbs, namely flash/onsights, worked clean routes and grade breakthroughs.

# Tag climbs that were flash/onsight
df_usendm.loc[df_usendm['Lead Style'].isin(CLEAN_SEND_FIRST), 'Flash/Onsight'] = True

# Create column that flags climbs that were worked. There are three possibilities to consider. We want 1 and 2.
# 1. Worked to clean send, no further sends.
# 2. Worked to clean send, additional attempts.
# 3. Sent clean first try, additional attempts.
df_all_dupes = df_usendm[df_usendm.duplicated(subset="Route ID", keep=False)] # First we filter for all duplicate entries.
df_all_worked = df_all_dupes.groupby('Route ID').filter(lambda x: ~x['Lead Style'].isin(CLEAN_SEND_FIRST).any()) # Then we remove all groups which have a lead style of flash or onsight to eliminate group 3.
df_worked_sub1 = df_all_worked[df_all_worked.groupby('Route ID')['Lead Style'].apply(lambda x: x.isin(CLEAN_SEND_WORKED))] # fell/hungs and TRs remain, so we take ticks from CLEAN_SEND_WORKED.
df_worked_sub2 = df_worked_sub1.loc[df_worked_sub1.groupby('Route ID')['Date'].idxmin()] # Use only the earliest redpoint to correctly identify the first redpoint.
df_usendm.loc[df_worked_sub2.index.values, "Worked Clean"] = True

# Flag grade breakthrough ticks
dfbreakthr = df_usendm[(df_usendm['Flash/Onsight'] == True) | (df_usendm['Worked Clean'] == True)]
breakthrough_indexes = dfbreakthr.groupby('Rating')['Date'].idxmin().values
df_usendm.loc[breakthrough_indexes, "Grade Breakthrough"] = True
df_usendm.loc[breakthrough_indexes]

# Count number of attempts to send
# Assumes no style lead ticks are fell/hung
# Assumes rp/pp with no prior tick history has one prior attempt
# Counts clean ticks with multiple pitches as total attempts. It also counts fell/hung, and TR with multiple pitches as multiple attempts.
# !!! This will falsely identify a single pitch climb broken into multiple pitches as two attempts, there isn't really a good way to detect this.
df_all_worked.loc[df_worked_sub2.index.values, "Worked Clean"] = True # To make the next line work properly, we need to backfill the "worked clean" flag to worked clean.
df_all_worked.drop(df_all_worked[(df_all_worked['Lead Style'].isin(CLEAN_SEND_WORKED)) & (df_all_worked['Worked Clean'] == '')].index, inplace=True) # remove all redpoints/pinkpoints after first for purpose of counting attempts, this can be done by removing all rows that are redpoints but not marked 'worked clean'
num_to_send = df_all_worked.groupby('Route ID')['Pitches'].sum()
matched_attempts = df_usendm[df_usendm['Worked Clean'] == True].merge(num_to_send, on="Route ID", how="left")
matched_attempts.index = df_usendm[df_usendm['Worked Clean'] == True].index # I'm dumb and this is the best way I could find to get my index to remain
df_usendm.loc[matched_attempts.index, "Attempts"] = matched_attempts.iloc[:,-1]
df_usendm.loc[df_usendm['Attempts'] == 1, 'Attempts'] = 2 # This assumes rp with 1 pitch and no prior ticks had one prior attempt

C:\Users\Brayden\AppData\Local\Temp\ipykernel_13844\133803154.py:12: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



Plot and visualize

In [409]:
# First we create a copy of the user modified dataframe, which we will refer to as the filtered dataframe
df_usendf = df_usendm.copy()

# Filters, these would be sliders and options in an interactive plot
roped_grade_min = '5.10a'
boulder_grade_min = 'V0'

# We slice the dataframe into a route and a boulder type. Each type will receive it's own independent filtering.

df_usendf_r = df_usendf[df_usendf['Rating'].isin(YDS_GRADES_FULL[YDS_GRADES_FULL.index(roped_grade_min):])] # the : here is key, it makes it so that it includes all grades above the min included.
df_usendf_b = df_usendf[df_usendf['Rating'].isin(V_GRADES_FULL[V_GRADES_FULL.index(boulder_grade_min):])]

In [410]:
# Create dataframe of clean sends for analysis

df_usendf_r.loc[df_usendf_r['Lead Style'].isin(CLEAN_SEND_FIRST), "Attempts"] = '' # Optionally change attempts from blank to 1 or other
df_clean_sends_r = df_usendf_r[(df_usendf_r['Lead Style'].isin(CLEAN_SEND))]
# df_clean_sends_r = df_clean_sends_r.loc[df_clean_sends_r.groupby('Route ID')['Date'].idxmin()] # Optionally ignore subequent clean sends
df_clean_sends_r['Date Formatted'] = df_clean_sends_r['Date'].dt.date

df_usendf_b.loc[df_usendf_b['Lead Style'].isin(CLEAN_SEND_FIRST), "Attempts"] = '' # Optionally change attempts from blank to 1 or other
df_clean_sends_b = df_usendf_b[(df_usendf_b['Style'].isin(CLEAN_SEND))]
# df_clean_sends_b = df_clean_sends_b.loc[df_clean_sends_r.groupby('Route ID')['Date'].idxmin()] # Optionally ignore subequent clean sends
df_clean_sends_b['Date Formatted'] = df_clean_sends_b['Date'].dt.date

C:\Users\Brayden\AppData\Local\Temp\ipykernel_13844\3433391245.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Brayden\AppData\Local\Temp\ipykernel_13844\3433391245.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [411]:
# Set visualization settings accordant to modifications and filters desired.

if r_grade_homo_type == '1':
    ryaxorder = YDS_GRADES_LETTER
else:
    ryaxorder = YDS_GRADES_SIGN

if b_grade_homo_type == '1':
    byaxorder = V_GRADES_BASIC
else:
    byaxorder = V_GRADES_SIGN

In [412]:
fig = px.bar(df_clean_sends_r, y="Rating", orientation='h', category_orders={"Rating": ryaxorder[::-1]}, text='Attempts', custom_data=['Route', 'Date Formatted', 'Location', 'Length', 'Avg Stars']) # The [::-1] is an inverse slice
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':20}, title={'text':'<b>Climbing Pyramid</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Number of Routes Sent'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_color='#7A4F25', textposition = "inside", textfont={"color": 'White', "size": 12, "family": 'Arial Black'},  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')
# fig.update_traces(marker_color=list(map(lambda x: '#7A4F25' if (x=='') else '#bf9315', df_clean_sends['Attempts'])), textposition = "inside",  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')

In [413]:
fig = px.scatter(df_clean_sends_r, "Date", "Rating", category_orders={"Rating": ryaxorder[::-1]}, text='Attempts', custom_data=['Route', 'Date Formatted', 'Location', 'Length', 'Avg Stars'])
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':20}, title={'text':'<b>Send by Date</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Date'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_symbol='square', marker_color='#7A4F25', marker_size=20, marker_line_width=2, marker_line_color='black', textfont={"color": 'White', "size": 12}, hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')

In [414]:
fig = px.bar(df_clean_sends_b, y="Rating", orientation='h', category_orders={"Rating": byaxorder[::-1]}, text='Attempts', custom_data=['Route', 'Date Formatted', 'Location', 'Length', 'Avg Stars']) # The [::-1] is an inverse slice
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':18}, title={'text':'<b>Climbing Pyramid</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Number of Problems Sent'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_color='#7A4F25', textposition = "inside", textfont={"color": 'White', "size": 12, "family": 'Arial Black'},  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')
# fig.update_traces(marker_color=list(map(lambda x: '#7A4F25' if (x=='') else '#bf9315', df_clean_sends['Attempts'])), textposition = "inside",  hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')

In [416]:
fig = px.scatter(df_clean_sends_b, "Date", "Rating", category_orders={"Rating": byaxorder[::-1]}, text='Attempts', custom_data=['Route', 'Date Formatted', 'Location', 'Length', 'Avg Stars'])
fig.update_layout(font={'family':'Courier New', 'color':'black', 'size':20}, title={'text':'<b>Send by Date</b>', 'x':0.5, 'font_size':30}, xaxis={'title': 'Date'}, yaxis={'title': 'Grade'}, paper_bgcolor='#ece5dc', plot_bgcolor='#F5D3A5', bargap=0)
fig.update_traces(marker_symbol='square', marker_color='#7A4F25', marker_size=20, marker_line_width=2, marker_line_color='black', textfont={"color": 'White', "size": 12}, hovertemplate='Name: %{customdata[0]}<br>Date: %{customdata[1]}<br>Location: %{customdata[2]}<br>Length: %{customdata[3]}ft<br>Avg Stars: %{customdata[4]}')